In [ ]:
!pip install ccxt pandas ta pytz matplotlib ipywidgets

In [ ]:
import ccxt
import pandas as pd
import ta
import pytz
import matplotlib.pyplot as plt
from datetime import datetime
import time

exchange = ccxt.kraken()
market = 'BTC/USD' if 'BTC/USD' in exchange.symbols else 'XBT/USD'

def fetch_ohlcv(timeframe='1h', limit=500):
    ohlcv = exchange.fetch_ohlcv(market, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df['timestamp'] = df['timestamp'].dt.tz_convert('America/Denver')
    return df.set_index('timestamp')

def add_indicators(df):
    df['SMA50'] = df['close'].rolling(50).mean()
    df['SMA200'] = df['close'].rolling(200).mean()
    df['RSI'] = ta.momentum.rsi(df['close'], window=14)
    macd = ta.trend.macd(df['close'], window_slow=26, window_fast=12)
    macd_signal = ta.trend.macd_signal(df['close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD'] = macd
    df['MACD_signal'] = macd_signal
    return df

def generate_signal(df):
    return 'Bullish' if df['SMA50'].iloc[-1] > df['SMA200'].iloc[-1] else 'Bearish'

def run_once(timeframe='1h'):
    df = fetch_ohlcv(timeframe)
    df = add_indicators(df)
    signal = generate_signal(df)
    now = datetime.now(pytz.timezone('America/Denver')).strftime('%Y-%m-%d %H:%M:%S')
    print(f"{now} - {signal}")
    return df

def live_watch(timeframe='1h', loops=60, sleep_s=60):
    for _ in range(loops):
        run_once(timeframe)
        time.sleep(sleep_s)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

tf_dropdown = widgets.Dropdown(options=['5m','15m','1h'], value='1h', description='Timeframe:')
run_btn = widgets.Button(description='Run snapshot')
live_btn = widgets.Button(description='Start live')

def on_run(btn):
    df = run_once(tf_dropdown.value)
    display(df.tail())

def on_live(btn):
    live_watch(tf_dropdown.value, loops=60, sleep_s=60)

run_btn.on_click(on_run)
live_btn.on_click(on_live)

display(tf_dropdown, run_btn, live_btn)


In [ ]:
df = run_once('1h')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
ax1.plot(df.index, df['close'], label='Price')
ax1.plot(df.index, df['SMA50'], label='SMA50')
ax1.plot(df.index, df['SMA200'], label='SMA200')
ax1.legend()
ax1.set_title('BTC Price with SMA50/200')
ax2.plot(df.index, df['RSI'], label='RSI')
ax2.axhline(30, color='red', linestyle='--')
ax2.axhline(70, color='red', linestyle='--')
ax2.set_title('RSI')
plt.show()
